# 합성곱 신경망 - CNN(Convolutional Neural Network)

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'New Gulim'
rcParams['font.size'] = 10
rcParams['axes.unicode_minus'] = False

# 1 CNN 전체 구조
- 합성곱 계층(Convolutional Layer)
- 풀링 계층(Pooling Layer)
- 완전 연결 계층(Fully Connected Layer) - Affine 계층
- ReLU 계층(ReLU Layer)
- 소프트맥스 계층(Softmax Layer)

- 완전 연결 계층(Affine 계층)으로 이루어진 네트워크
<img src="./images/fig_7-1.png" width="600"/>

- CNN으로 이루어진 네트워크: 합성곱 계층과 풀링 계층이 추가
<img src="./images/fig_7-2.png" width="600"/>

# 2 합성곱 계층 - Convolutional Layer

### 2.1 합성곱 연산

- 합성곱 연산의 예
<img src="./images/fig_7-3.png" width="500"/>

- 합성곱 연산의 계산 순서
<img src="./images/fig_7-4.png" width="400"/>

- 합성곱 연산의 편향: 필터를 적용한 원소에 고정값(편향)을 더한다
<img src="./images/fig_7-5.png" width="600"/>

### 2.2 패딩 - padding

- 합성곱 연산의 패딩 처리: 입력 데이터 주위에 0을 채운다(점선 영역)
<img src="./images/fig_7-6.png" width="500"/>

### 2.3 스트라이드 - stride

- 스트라이드가 2인 합성곱 연산
<img src="./images/fig_7-7.png" width="500"/>

- 출력 크기 계산
- 입력 크기(H, W), 필터 크기(FH, FW), 출력 크기(OH, OW)
<img src="./images/e_7.1.png" width="200"/>

- 출력 크기 계산 예제
<img src="./images/fig_7-7-1.png" width="400"/>

### 2.4 3차원 데이터의 합성곱 연산

- 3차원 데이터 합성곱 연산의 예
<img src="./images/fig_7-8.png" width="500"/>

- 3차원 데이터 합성곱 연산의 계산 순서
<img src="./images/fig_7-9.png" width="400"/>

### 2.5 블록으로 표현

- 합성곱 연산으로 블록으로 표현
<img src="./images/fig_7-10.png" width="500"/>

- 여러 필터를 사용한 합성곱 연산의 예: 출력이 다수의 채널로 생성
<img src="./images/fig_7-11.png" width="500"/>

- 합성곱 연산의 처리 흐름(편향 추가)
<img src="./images/fig_7-12.png" width="600"/>

### 2.6 배치 처리

- 합성곱 연산의 처리 흐름(배치 처리)
<img src="./images/fig_7-13.png" width="600"/>

# 3 풀링 계층 - Pooling Layer

- 최대 풀링(Max Pooling)의 처리 순서
<img src="./images/fig_7-14.png" width="600"/>

### 3.1 풀링 계층의 특징

- 풀링 윈도우 크기와 스트라이드는 동일하게 설정한다
- 학습해야 할 매개변수가 없다(필터 없음)
- 채널 수가 변하지 않는다
- 입력의 변화에 영향을 적게 받는다(강건하다)

- 풀링은 채널 수를 바꾸지 않는다
<img src="./images/fig_7-15.png" width="400"/>

- 입력 데이터가 가로로 1원소만큰 어긋나도 출력은 같다
<img src="./images/fig_7-16.png" width="600"/>

# 4 합성곱/풀링 계층 구현

### 4.1 CNN(4차원 데이터)
- (N, C, H, W)
- (데이터수, 채널수, 높이, 너비)

### 4.2 데이터 전개 - im2col 트릭 사용

- 4차원 데이터를 2차원 데이터로 변환

- im2col의 동작
<img src="./images/fig_7-17.png" width="400"/>

- 필터 적용 영역을 앞에서부터 순서대로 1줄로 펼친다
<img src="./images/fig_7-18.png" width="400"/>

- 합성곱 연산의 필터 처리 과정
<img src="./images/fig_7-19.png" width="500"/>

### 4.3 합성곱 계층 구현

#### 4.3.1 im2col 함수 사용 예제

##### im2col(input_data, filter_h, filter_w, stride=1, pad=0)
- input_data - (데이터수, 채널수, 높이, 너비)의 4차원 배열
- filter_h - 필터의 높이
- filter_w - 필터의 너비
- stride - 스트라이드
- pad - 패팅

In [ ]:
from common.util import im2col

x1 =  np.random.rand(1, 3, 7, 7) # (데이터수/ 채널수/ 높이/ 너비)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape) # (9, 75)

x2 = np.random.rand(10, 3, 7, 7) # 데이터 10개
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape) # (90, 75)

#### 4.3.2 합성곱 계층 코드 구현

- 축 순서 변경: numpy.transpose() 사용
<img src="./images/fig_7-20.png" width="500"/>

In [ ]:
# 합성곱 계층 코드 구현

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # 필터 전개
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

### 4.4 풀링 계층 구현

- 입력 데이터에 풀링 적용 영역을 전개(2X2 풀링의 예)
<img src="./images/fig_7-21.png" width="400"/>

- 풀링 계층 구현의 흐름: 풀링 적용 영역에서 가장 큰 원소는 회색으로 표시
<img src="./images/fig_7-22.png" width="600"/>

In [ ]:
# 풀링 계층 코드 구현

class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 전개(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 최대값(2)
        out = np.max(col, axis=1)
        
        # 성형(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

# 5 CNN 구현

### 5.1 CNN class 코드 구현

- 단순한 CNN 네트워크 구성
<img src="./images/fig_7-23.png" width="600"/>

##### SimpleConvNet() 생성시 입력 변수
- input_dim - 입력 데이터(채널수, 높이, 너비)의 차원
- conv_param - 합성곱 계층의 하이퍼파라미터(딕셔너리). 딕셔너리의 키는 다음과 같다
 - filter_num - 필터수
 - filter_size - 필터크기
 - stride - 스트라이드
 - pad - 패딩
- hidden_size - 은닉층(완전연결)의 뉴런수
- output_size - 출력층(완전연결)의 뉴런수
- weight_init_std - 초기화 때의 가중치 표준편차

In [ ]:
# SimpleConvNet() 코드 구현

import pickle
from collections import OrderedDict
from common.layers import *
from common.gradient import numerical_gradient

class SimpleConvNet:
    """단순한 합성곱 신경망
    
    conv - relu - pool - affine - relu - affine - softmax
    
    Parameters
    ----------
    input_size : 입력 크기（MNIST의 경우엔 784）
    hidden_size_list : 각 은닉층의 뉴런 수를 담은 리스트（e.g. [100, 100, 100]）
    output_size : 출력 크기（MNIST의 경우엔 10）
    activation : 활성화 함수 - 'relu' 혹은 'sigmoid'
    weight_init_std : 가중치의 표준편차 지정（e.g. 0.01）
        'relu'나 'he'로 지정하면 'He 초깃값'으로 설정
        'sigmoid'나 'xavier'로 지정하면 'Xavier 초깃값'으로 설정
    """
    def __init__(self, input_dim=(1, 28, 28), 
                 conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                 hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))

        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
                            np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
                            np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * \
                            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                           conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

        self.last_layer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self, x, t):
        """손실 함수를 구한다.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블
        """
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads

    def gradient(self, x, t):
        """기울기를 구한다(오차역전파법).

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
        grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads
        
    def save_params(self, file_name="params.pkl"):
        params = {}
        for key, val in self.params.items():
            params[key] = val
        with open(file_name, 'wb') as f:
            pickle.dump(params, f)

    def load_params(self, file_name="params.pkl"):
        with open(file_name, 'rb') as f:
            params = pickle.load(f)
        for key, val in params.items():
            self.params[key] = val

        for i, key in enumerate(['Conv1', 'Affine1', 'Affine2']):
            self.layers[key].W = self.params['W' + str(i+1)]
            self.layers[key].b = self.params['b' + str(i+1)]

### 5.2 MNIST 데이터 학습 - SimpleConvNet() 사용

In [ ]:
%%time

from dataset.mnist import load_mnist
from common.trainer import Trainer

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
x_train, t_train = x_train[:5000], t_train[:5000]
x_test, t_test = x_test[:1000], t_test[:1000]

# 시간이 오래 걸릴 경우 max_epochs를 줄인다.
#max_epochs = 5
max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params('data/params.pkl')
print('Saved Network Parameters!')

In [ ]:
# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

# 6 CNN 시각화

### 6.1 CNN 1번째 층의 가중치 시각화

#####  가중치(필터) 시각화 함수 정의

In [ ]:
# 가중치(필터) 시각화 함수 정의

def filter_show(filters, nx=8, margin=3, scale=10):
    """
    c.f. https://gist.github.com/aidiary/07d530d5e08011832b12#file-draw_weight-py
    """
    FN, C, FH, FW = filters.shape
    ny = int(np.ceil(FN / nx))

    fig = plt.figure(figsize=(10,4))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

    for i in range(FN):
        ax = fig.add_subplot(ny, nx, i+1, xticks=[], yticks=[])
        ax.imshow(filters[i, 0], cmap=plt.cm.gray_r, interpolation='nearest')
    plt.show()

#####  가중치(필터) 시각화: 무작위(랜덤) 초기화 후의 가중치
- filter = (30, 1, 5, 5): 5X5(흑백) 30개의 필터

In [ ]:
network_v = SimpleConvNet()

filter_show(network_v.params['W1'])

#####  가중치(필터) 시각화: 학습된 가중치
- filter = (30, 1, 5, 5): 5X5(흑백) 30개의 필터

In [ ]:
network_v.load_params('data/params.pkl')

filter_show(network_v.params['W1'])

- 가로 에지와 세로 에지에 반응히는 필터
- 출력 이미지 1 은 세로 에지에 흰 픽셀이 나타나고, 출력 이미지 2 는 가로 에지에 흰 픽셀이 많이 나온다
 
<img src="./images/fig_7-25.png" width="700"/>

### 6.2 CNN 층 깊이에 따른 추출 정보 변화

- CNN의 합성곱 계층에서 추출되는 정보
 - 1 번째 층은 에지와 블롭
 - 3번째 층은 텍스처
 - 5번째 층은 사물의 일부
 - 마지막 완전연결 계층은 사물의 클래스(개, 자동차등)에 뉴린이 반응한다

<img src="./images/fig_7-26.png" width="700"/>

# 7 대표적인 CNN

### 7.1 LeNet

- 1998년에 제안된 첫번째 CNN
- (합성곱 계층 + 풀링 계층) 반복 + 완전 연결 계층
- 활성화 함수: Sigmoid 함수 사용 vs ReLU
- 풀링: 서브샘플링(단순히 원소의 수를 줄이기만 하는 용도) vs Max Pooling

- LeNet의 구성
<img src="./images/fig_7-27.png" width="700"/>

### 7.2 AlexNet

- 2012년에 제안된 CNN: 딥러닝 열풍의 주역
- (합성곱 계층 + 풀링 계층) 반복 + 완전 연결 계층
- 활성화 함수: ReLU 함수 사용
- 풀링: Max Pooling
- 드롭아웃 사용
- LRN(Local Response Normalization 이라는 국소적 정규화를 실시하는 계층 이용
- 빅데이터와 GPU의 적극적인 활용 가능

- AlexNet의 구성
<img src="./images/fig_7-28.png" width="600"/>

# 정리

- CNN은 지금까지의 완전연결 계층 네트워크에 합성곱 계층과 풀링 계층을 새로 추가한다.
- 합성곱 계층과 풀링 계층은 im2col(이미지를 행렬로 전개하는 함수)을 이용하면 간단하고 효율적으로 구현할수있다.
- CNN을 시각화 해보면 계층이 갚어질수록 고급 정보가 추출되는 모습을 확인할수 있다.
- 대표적인 CNN에는 LeNet과 AlexNet이 있다.
- 답러닝의 발전에는 빅데이터와 GPU가 크게 기여했다.

---

In [ ]:
# End of file